In [2]:
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# Load all datasets into a dictionary
dataset_paths = {
    'Rental Index': '/Users/joseignacio/Desktop/Project_3/df_rental_index_avg.csv',
    'Value Index': '/Users/joseignacio/Desktop/Project_3/df_value_index_avg.csv',
    'Market Index': '/Users/joseignacio/Desktop/Project_3/df_market_index_avg.csv',
    'New Construction Count': '/Users/joseignacio/Desktop/Project_3/df_newcon_count_avg.csv',
    'New Construction Sales': '/Users/joseignacio/Desktop/Project_3/df_newcon_sales_avg.csv',
    'Days Pending': '/Users/joseignacio/Desktop/Project_3/df_days_pending_avg.csv',
}

def load_dataset(path):
    df = pd.read_csv(path)
    df = df.drop(columns=['_id', 'RegionID', 'SizeRank', 'RegionType'], errors='ignore')
    return df

dataframes = {name: load_dataset(path) for name, path in dataset_paths.items()}

app = Dash(__name__)

app.layout = html.Div([
    html.H1("City vs US Average Comparison - All Datasets"),

    html.Label("Select Dataset:"),
    dcc.Dropdown(
        id='dataset-dropdown',
        options=[{'label': name, 'value': name} for name in dataframes.keys()],
        value='Rental Index'
    ),

    html.Label("Select Year:"),
    dcc.Slider(
        id='year-slider',
        min=2015, max=2025, step=1,
        marks={year: str(year) for year in range(2015, 2026)},
        value=2024
    ),

    html.Br(),

    html.Div([
        html.Label("Number of Cities to Display:"),
        dcc.Slider(
            id='city-count-slider',
            min=10,
            max=100,
            step=10,
            marks={i: str(i) for i in range(10, 110, 10)},
            value=50
        ),
        html.Label("Sort Order:"),
        dcc.RadioItems(
            id='sort-order-radio',
            options=[
                {'label': 'Descending', 'value': 'desc'},
                {'label': 'Ascending', 'value': 'asc'}
            ],
            value='desc',
            labelStyle={'display': 'inline-block', 'margin-right': '15px'}
        )
    ]),

    dcc.Graph(id='all-cities-bar'),

    html.Label("Select City:"),
    dcc.Dropdown(id='city-dropdown', placeholder="Select a city"),
    dcc.Graph(id='comparison-bar'),

    html.H2("Cities in State vs US Average"),
    html.Div([
        html.Label("Select State:"),
        dcc.Dropdown(id='state-dropdown', placeholder="Select a state"),

        html.Label("Sort Order:"),
        dcc.RadioItems(
            id='state-sort-order',
            options=[
                {'label': 'Descending', 'value': 'desc'},
                {'label': 'Ascending', 'value': 'asc'}
            ],
            value='desc',
            labelStyle={'display': 'inline-block', 'margin-right': '15px'}
        )
    ]),
    dcc.Graph(id='state-cities-bar')
])

@app.callback(
    [Output('city-dropdown', 'options'),
     Output('state-dropdown', 'options')],
    Input('dataset-dropdown', 'value')
)
def update_dropdowns(dataset_name):
    df = dataframes[dataset_name]
    city_options = [{'label': c, 'value': c} for c in df['RegionName'].dropna().unique() if c != 'United States']
    state_options = [{'label': s, 'value': s} for s in df['StateName'].dropna().unique()]
    return city_options, state_options

@app.callback(
    Output('comparison-bar', 'figure'),
    [Input('dataset-dropdown', 'value'),
     Input('city-dropdown', 'value'),
     Input('year-slider', 'value')]
)
def update_comparison_bar(dataset_name, city, year):
    df = dataframes[dataset_name]
    year = str(year)
    if not city or year not in df.columns:
        return px.bar(title="Select a city and valid year")

    us_val = df[df['RegionName'] == 'United States'][year].values[0]
    city_val = df[df['RegionName'] == city][year].values[0]
    fig = px.bar(
        pd.DataFrame({'Entity': [city, 'United States'], 'Value': [city_val, us_val]}),
        x='Entity', y='Value', title=f"{city} vs US Average in {year} ({dataset_name})",
        color_discrete_sequence=['#5DADE2']
    )
    return fig

@app.callback(
    Output('all-cities-bar', 'figure'),
    [Input('dataset-dropdown', 'value'),
     Input('year-slider', 'value'),
     Input('city-count-slider', 'value'),
     Input('sort-order-radio', 'value')]
)
def update_all_cities(dataset_name, year, count, order):
    df = dataframes[dataset_name]
    year = str(year)
    if year not in df.columns:
        return px.bar(title="Invalid year")

    df = df[df['RegionName'] != 'United States'][['RegionName', year]].dropna()
    df = df.sort_values(by=year, ascending=(order == 'asc')).head(count)
    us_val = dataframes[dataset_name][dataframes[dataset_name]['RegionName'] == 'United States'][year].values[0]
    fig = px.bar(df, x='RegionName', y=year, title=f"Top {count} Cities vs US Average ({year})",
                 color_discrete_sequence=['#5DADE2'])
    fig.add_hline(y=us_val, line_dash="dot", line_color="red", annotation_text="US Average")
    fig.update_layout(xaxis_tickangle=60, height=600, width=1400)
    return fig

@app.callback(
    Output('state-cities-bar', 'figure'),
    [Input('dataset-dropdown', 'value'),
     Input('year-slider', 'value'),
     Input('state-dropdown', 'value'),
     Input('state-sort-order', 'value')]
)
def update_state_chart(dataset_name, year, state, sort_order):
    df = dataframes[dataset_name]
    year = str(year)
    if not state or year not in df.columns:
        return px.bar(title="Select a state and valid year")

    df = df[(df['StateName'] == state) & (df['RegionName'] != 'United States')][['RegionName', year]].dropna()
    df = df.sort_values(by=year, ascending=(sort_order == 'asc'))
    us_val = dataframes[dataset_name][dataframes[dataset_name]['RegionName'] == 'United States'][year].values[0]
    fig = px.bar(df, x='RegionName', y=year, title=f"Cities in {state} vs US Average ({year})",
                 color_discrete_sequence=['#5DADE2'])
    fig.add_hline(y=us_val, line_dash="dot", line_color="red", annotation_text="US Average")
    fig.update_layout(xaxis_tickangle=60, height=600, width=1400)
    return fig

if __name__ == '__main__':
    app.run(debug=True)